main.py

In [ ]:
# # -*- coding: utf-8 -*-
import numpy as np
import pandas as pd



import tensorflow as tf



## Final model Class define
class Final_Model:
    def __init__(self):
        """
        Final_Model constructer
        """
        self.model = self.get_model()
        self.model.load_weights("RESNET50.h5")
    
    def predict(self,image):
        """
        This function return predicted value

        Input:
            image: 320x320x3 size array
        Output:
            Pred: predicted dimensions (x_center,y_center,box oriantation,box height,box width) 
            Edges: 4x2 array, contains points of corner
        """

        height, width = image.shape[:2]
        tt,ttt =  1,1
        T = np.float32([[1, 0, 50 * tt], [0, 1, 50 * ttt]])
        image = cv2.warpAffine(image, T, (width, height))

        pred = self.model.predict(np.array([image]))[0]
        edges = self.grasp_to_bbox_pred(pred[0], pred[1],pred[2],pred[3],pred[4]) - np.array([50,50])

        return pred, edges

    def grasp_to_bbox_pred(self,x, y, angle, h, w):
        """
        This function return corner points using given dimensions (x_center,y_center,box oriantation,box height,box width) 

        Input:
            x: x cordinate of box center
            y: y cordinate of box center
            angle: Oriantation of box respect to positive x-axis
            h: height of box.
            w: width of box
        Output:
            Edges: 4x2 array, contains points of corner
        """
        edge1 = (x - w/2, y + h/2)
        edge2 = (x + w/2, y + h/2)
        edge3 = (x + w/2, y - h/2)
        edge4 = (x - w/2, y - h/2)
        a = np.array([edge1, edge2, edge3, edge4])
        a = a - (x,y)
        angle = angle * (3.14/180)
        fun = lambda x: (x[:,0] * np.cos(angle) - x[:,1] * np.sin(angle), x[:,0] * np.sin(angle) + x[:,1] * np.cos(angle))
        a = fun(a)
        a = np.array(a).T
        a = a + (x,y)
        return a

    def plot_predict(self,image):
        """
        This function return predicted values and plot predicted values on image

        Input:
            image: 320x320x3 size array
        Output:
            Pred: predicted dimensions (x_center,y_center,box oriantation,box height,box width) 
            Edges: 4x2 array, contains points of corner
        """
        plot_im = image

        height, width = image.shape[:2]
        tt,ttt =  1,1
        T = np.float32([[1, 0, 50 * tt], [0, 1, 50 * ttt]])
        image = cv2.warpAffine(image, T, (width, height))

        pred = self.model.predict(np.array([image]))[0]
        edges = self.grasp_to_bbox_pred(pred[0], pred[1],pred[2],pred[3],pred[4])

        x = edges[:,0].reshape(1,-1)[0]
        y = edges[:,1].reshape(1,-1)[0]
        
        x = np.concatenate([x,x[:1]]) - 50
        y = np.concatenate([y,y[:1]]) - 50

        fig, ax = plt.subplots(figsize = (8,8))
        ax.imshow(plot_im)
        ax.plot(x,y, 'ro-',color = 'green')

        return pred,edges


    def get_model(self,lr = 0.001):

        """
        This function load trained model and return it.

        Input: 
            lr: learning rate but its not require here.
        Output:
            model: return trained model, type: tf.keras.Model
        """

        def resenet_load():
            RESNET50 = tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet',input_shape=(320,320,3), classes=2048)
            for i in range(len(RESNET50.layers)):
                RESNET50.layers[i].trainable = False
            return RESNET50

        RESNET50 = resenet_load()

        input = tf.keras.layers.Input((320,320,3,))
        resnet = RESNET50(input)
        x = tf.keras.layers.Flatten()(resnet)
        x = tf.keras.layers.Dense(1024,activation='relu')(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        output = tf.keras.layers.Dense(5)(x)
        model = tf.keras.models.Model(input,output)
    
        model.compile(tf.keras.optimizers.Adam(lr), loss = 'mse')
        return model


model = Final_Model()


from flask import Flask, render_template, request, redirect, url_for
import cv2
import numpy
import matplotlib.pyplot as plt
import os
import random
import time


app = Flask(__name__,template_folder='templates')

@app.route('/')
def index():
    return render_template('index.html')



@app.route('/predict',methods=['POST'])
def predict():
    filestr = request.files['file'].read()
    
    i = random.randint(1,10000000)
    
    npimg = numpy.fromstring(filestr, numpy.uint8)
    img = cv2.imdecode(npimg, cv2.IMREAD_COLOR)
    (x, y, angle, h, w),edges = model.predict(img)
    x = edges[:,0].reshape(1,-1)[0]
    y = edges[:,1].reshape(1,-1)[0]
    
    x = np.concatenate([x,x[:1]])
    y = np.concatenate([y,y[:1]])
    
    plt.imshow(img)
    plt.plot(x,y, 'ro-',color = 'green')
    plt.axis('off')
    plt.tick_params(axis='both', left='off', top='off', right='off', bottom='off', labelleft='off', labeltop='off', labelright='off', labelbottom='off')
    plt.savefig('./static/plot'+str(i)+'.png', dpi=100, bbox_inches='tight', pad_inches=0.0)
    time.sleep(1)
    plt.clf()

    return render_template('index.html', filename='plot'+str(i)+'.png')


if __name__ == '__main__':
    app.run(debug=True)

index.html

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Robort Grasp Detection</title>

    <style>
        h1 {
            
            margin-left: auto;
            margin-right: auto;
        }
        textarea {
            width:350px;
            height:150px;
        }

        .button {
            background-color: #4CAF50; /* Green */
            border: none;
            color: white;
            padding: 15px 32px;
            text-align: center;
            text-decoration: none;
            display: inline-block;
            font-size: 16px;
            margin-top: 5px;
          }

         
    </style>
</head>
<body>
    <h1>
        Robort Grasp Detection
    </h1>
   <form action='/predict' method='post' enctype="multipart/form-data">
    <div>
        <label for="myfile">Select a file:</label>
        <input type="file" id="file" name="file">
    </div>
    <div>
       <input class="button" type='submit' value='Predict'>
    </div>
   </form> 

   {% if filename %}
   <div>
    
    <img src="{{url_for('static',filename=filename)}}" />
   </div>
   {% endif %}
   
</body>
</html>

app.yml

In [ ]:
runtime: python
threadsafe: yes
env: flex
entrypoint: gunicorn -b :$PORT main:app

resources: 
  cpu: 8
  memory_gb: 16
  disk_size_gb: 30

runtime_config:
  python_version: 3

requirements.txt

In [ ]:
numpy==1.19.5
pandas==1.1.5
tensorflow==2.5.0
nltk==3.2.5
gunicorn==19.9.0
Flask==1.1.1
matplotlib==3.2.2
opencv-python--headless